# **Predictive Crime Mapping with a spatio-temporal kernel density estimation approach**

## **Case of study: Dallas City**

*Institute of Mathematical and Computational Engineering*

*School of Engineering*

*Pontifical Catholic University of Chile*

**Authors:** Christian Carstens, Mauro Mendoza, Francisco Tobar

**Date:** 2019-04-11

`Python version: 3.7`

### **DB Request**

In [1]:
import pandas as pd
from sodapy import Socrata

import credentials as cre

In [17]:
with Socrata(cre.socrata_domain,
             cre.API_KEY_S,
             username=cre.USERNAME_S,
             password=cre.PASSWORD_S) as client:
    query = """
select
    incidentnum,
    geocoded_column,
    date1,
    time1
where
    geocoded_column is not null
    and date1 is not null
    and time1 is not null
limit
    1000
""" # 530000 max. 11/04

    results = client.get(cre.socrata_dataset_identifier, query=query, content_type='json')
    df = pd.DataFrame.from_records(results)

### **DB Cleaning & Formatting**

#### Cleaning

In [10]:
from datetime import date, time, datetime
from time import strftime

In [18]:
# 'date1' column cleaning: deleting 'T00:00:00.000'
# df["date1"] = df['date1'].apply(lambda x: date(*(int(i) for i in x.split('T')[0].split('-'))))

df["date1"] = df['date1'].apply(lambda x: '/'.join([str(int(i)) for i in x.split('T')[0].split('-')[::-1]]))
df["time1"] = df['time1'].apply(lambda x: datetime.strptime(x, "%H:%M").strftime("%I:%M %p"))

df['datetime'] = df["date1"] + ' ' + df["time1"]

# 'incidentnum' column cleaning: deleting '20XX'
df["incidentnum"] = df['incidentnum'].apply(lambda x: x.split('-')[0])

# Modificar la columna 'geocoded_column': Dividirla en dos columnas -> latitude, longitude

# dict to list: [latitude, longitude]
df['geocoded_column'] = df['geocoded_column'].apply(lambda x: x['coordinates'])

# Column creation: latitude & longitude
df['longitude'] = df['geocoded_column'].apply(lambda x: x[0])
df['latitude'] = df['geocoded_column'].apply(lambda x: x[1])

# Deletion the old column 'geocoded_column'
del df['geocoded_column']
del df["date1"]
del df["time1"]

#### Formatting

In [19]:
# Cambio de nombre en las columnas
df.rename(columns={'incidentnum':'incident_id'}, inplace=True)

# Cambio de las posiciones de las columnas
df = df[['incident_id', 'datetime',  'longitude', 'latitude']]

# Sort of the 'incident_id' column
df.sort_values(by=['incident_id'], inplace=True)

# Reset the old index
df.reset_index(drop=True, inplace=True)
df.index = df.index + 1

In [20]:
# En caso de que queramos exportar la base a .csv
# df.to_csv('dallas_db.csv', index=False)

### Dallas Map

Cargamos los elementos necesarios para la visualización del mapa.

In [21]:
from arcgis import GIS, geocoding, features, geometry
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor

from credentials import arcgis_domain, USERNAME_A, PASSWORD_A

In [22]:
gis = GIS(arcgis_domain, USERNAME_A, PASSWORD_A)

In [23]:
df.head()

,incident_id,datetime,longitude,latitude
1,000072,1/1/2019 02:23 AM,-96.678121,32.690058
2,000096,1/1/2019 12:45 AM,-96.884803,32.724354
3,001451,3/1/2018 01:00 AM,-96.894369,32.858183
4,001725,12/11/2018 04:50 PM,-96.797560,32.780990
5,002606,29/12/2017 11:30 PM,-96.792818,32.781247


In [26]:
map_1 = gis.map('Dallas', zoomlevel=11, mode="3D")
map_1.basemap = "dark-gray-vector"

sdf = df.spatial
incidents = gis.content.import_data(df, title="Dallas Incidents")

map_1.add_layer(incidents)

map_1

MapView(layout=Layout(height='400px', width='100%'), mode='3D', zoom=11.0)

Additional info. about Dallas City.

In [40]:
gcode = geocoding.geocode(address="Dallas, Texas")[0]["location"]
citylong = gcode["x"]
citylat = gcode["y"]

f"Dallas -> longitude: {citylong}, latitude: {citylat}"

'Dallas -> longitude: -96.79539999999997, latitude: 32.77815000000004'